Let $C = \{c_1, c_2, ..., c_n\}$ be the comments and $X_i$ = $x_i^j , j=1,2,\cdots,m$, where $x_i^j$ the annotation of annotator j for comment i.


Let $X^{\rho}_i$ the annotations for $c_i$ which belong in a partition $\rho \in \Rho$. Each $\rho$ in this case is a factor of feature $\Rho$

Then we can define $aposteriori(c_i) = max_{\rho} \{ndfu(X_i) - ndfu(X_i^ \rho)\}$
and $aposteriori(C) = Wilcoxon(aposteriori(c_i), 0, alternative="greater")$

In [2]:
%load_ext autoreload 
%autoreload 2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import re

from ndfu import ndfu
import aposteriori

In [3]:
df = pd.read_pickle("attitudes_embedded.csv")
df

,tweet,toyou,toany,annotatorMinority,bert,annotatorPolitics,traditionalism,annotatorRace,annotatorAge,annotatorGender,freeSpeech,harmHateSpeech,intent,lingPurism,racism,racist
0,""" you ugly bitch!!!""- that seems to be my real...","[2, 1, 1, 2]","[3, 3, 1, 2]","[{}, pansexual, disabled, trans, {}, {}]","[0.0047034588642418385, 0.11112377792596817, 0...","[-1.0, -1.0, 1.0, 0.5]","[3, 1, 5, 4]","[black, black, white, white]","[25.0, 35.0, 50.0, 25.0]","[man, man, woman, woman]","[1, 1, 1, 2]","[4, 5, 5, 2]","[1, 1, 1, 2]","[1, 1, 5, 5]","[1, 1, 1, 4]","[1, 1, 1, 1]"
1,"""""""I am happy to report that drowning does kil...","[4, 2, 1, 2, 1, 1]","[4, 2, 1, 2, 2, 1]","[{}, {}, {}, {}, {}, {}]","[0.35706740617752075, 0.032704465091228485, 0....","[-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]","[4, 3, 4, 3, 1, 4]","[black, black, white, white, white, white]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[woman, woman, woman, man, man, man]","[1, 1, 3, 4, 1, 1]","[4, 4, 3, 1, 5, 4]","[4, 2, 1, 3, 1, 1]","[2, 5, 5, 5, 5, 5]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 1, 1, 1, 2]","[2, 1, 1, 1, 1, 3]","[{}, black; female, nan, {}, {}, {}]","[-0.022276882082223892, 0.17076537013053894, 0...","[-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]","[3, 4, 3, 5, 1, 3]","[black, black, white, white, white, white]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[woman, woman, man, woman, man, woman]","[1, 1, 5, 1, 1, 1]","[4, 5, 1, 5, 5, 4]","[1, 1, 1, 1, 1, 1]","[5, 4, 5, 5, 5, 4]","[1, 1, 5, 1, 1, 1]","[1, 1, 1, 1, 1, 1]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 2, 1, 1, 1, 1]","[1, 4, 1, 1, 1, 1]","[{}, black; female, nan, {}, {}, nan]","[0.07474598288536072, 0.24487990140914917, 0.2...","[0.0, -1.0, 1.0, 0.5, -0.5, -1.0]","[4, 4, 3, 4, 5, 5]","[black, black, white, white, white, white]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[man, woman, man, woman, woman, woman]","[1, 1, 5, 3, 1, 1]","[4, 5, 1, 3, 5, 5]","[1, 1, 1, 1, 1, 1]","[4, 4, 5, 5, 4, 3]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 3, 3, 4]","[1, 2, 4, 4, 4]","[{}, NONE, No i do not, none, {}]","[0.050774428993463516, 0.08187347650527954, 0....","[-1.0, 1.0, 0.5, 0.5, -1.0]","[5, 3, 5, 5, 5]","[black, black, white, white, white]","[45.0, 45.0, 45.0, 60.0, 40.0]","[man, man, woman, woman, woman]","[1, 5, 1, 1, 1]","[5, 4, 3, 5, 2]","[1, 3, 1, 3, 5]","[5, 3, 4, 5, 5]","[1, 3, 5, 1, 1]","[1, 4, 2, 2, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 2, 3]","[4, 5, 2, 3]","[pansexual, disabled, trans, {}, none, {}]","[0.22457970678806305, 0.3840716481208801, -0.4...","[-1.0, 1.0, 0.5, 0.5]","[1, 5, 5, 5]","[black, black, white, white]","[35.0, 30.0, 45.0, 65.0]","[man, woman, man, woman]","[1, 5, 1, 1]","[5, 5, 5, 5]","[1, 5, 1, 3]","[1, 5, 5, 5]","[1, 5, 1, 1]","[1, 1, 1, 1]"
622,worst part about getting highlights is sitting...,"[1, 3, 1, 1, 1, 1]","[1, 5, 1, 1, 1, 1]","[{}, NONE, {}, {}, {}, none]","[0.1781517118215561, 0.02407737821340561, 0.13...","[-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]","[5, 3, 5, 2, 4, 3]","[black, black, white, white, white, white]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[man, man, woman, man, man, man]","[1, 5, 1, 2, 1, 1]","[5, 4, 4, 2, 3, 5]","[1, 3, 1, 1, 1, 1]","[5, 3, 5, 2, 5, 4]","[1, 5, 2, 3, 2, 1]","[1, 4, 1, 1, 1, 1]"
623,yids are not as honest as the rest of us,"[5, 5, 4, 5, 4, 1]","[5, 5, 4, 5, 4, 1]","[nan, {}, {}, none, {}, bisexual]","[0.2983386516571045, 0.31099164485931396, -0.4...","[-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]","[3, 5, 2, 5, 4, 3]","[black, black, white, white, white, white]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[man, man, man, woman, woman, woman]","[1, 1, 5, 1, 2, 1]","[3, 3, 2, 5, 3, 5]","[5, 5, 4, 5, 5, 1]","[5, 3, 5, 5, 4, 5]","[1, 1, 4, 1, 2, 1]","[5, 3, 3, 5, 5, 1]"
624,"you've got to be fucking joking, my room has l...","[3, 3, 1, 1, 2, 1]","[2, 4, 1, 2, 3, 2]","[

y=Racism, x={age, race, gender}

In [4]:
df = df.loc[:, ["tweet", "racism", "annotatorAge","annotatorRace", "annotatorGender"]]
df

,tweet,racism,annotatorAge,annotatorRace,annotatorGender
0,""" you ugly bitch!!!""- that seems to be my real...","[1, 1, 1, 4]","[25.0, 35.0, 50.0, 25.0]","[black, black, white, white]","[man, man, woman, woman]"
1,"""""""I am happy to report that drowning does kil...","[1, 1, 5, 5, 1, 1]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[black, black, white, white, white, white]","[woman, woman, woman, man, man, man]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 5, 1, 1, 1]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[black, black, white, white, white, white]","[woman, woman, man, woman, man, woman]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 1, 5, 5, 1, 1]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[black, black, white, white, white, white]","[man, woman, man, woman, woman, woman]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 5, 1, 1]","[45.0, 45.0, 45.0, 60.0, 40.0]","[black, black, white, white, white]","[man, man, woman, woman, woman]"
...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 1, 1]","[35.0, 30.0, 45.0, 65.0]","[black, black, white, white]","[man, woman, man, woman]"
622,worst part about getting highlights is sitting...,"[1, 5, 2, 3, 2, 1]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[black, black, white, white, white, white]","[man, man, woman, man, man, man]"
623,yids are not as honest as the rest of us,"[1, 1, 4, 1, 2, 1]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[black, black, white, white, white, white]","[man, man, man, woman, woman, woman]"
624,"you've got to be fucking joking, my room has l...","[1, 4, 1, 3, 3, 1]","[40.0, 25.0, 40.0, 35.0, 45.0, 35.0]","[black, black, white, white, white, white]","[woman, woman, woman, woman, woman, woman]"


In [5]:
def find_inconsistent_rows(df):
    inconsistent_rows = []
    
    for index, row in df.iterrows():
        list_lengths = [len(row[col]) for col in df.columns if isinstance(row[col], list)]
        
        if len(set(list_lengths)) > 1:  # Check if lengths are inconsistent
            inconsistent_rows.append(index)
    
    return df.loc[inconsistent_rows]


find_inconsistent_rows(df)

,tweet,racism,annotatorAge,annotatorRace,annotatorGender


## Annotator analysis

### Gender

In [6]:
def extract_annotations_and_attributes(
    df: pd.DataFrame, annot_col: str, attribute_col: str
) -> tuple[list, list]:
    all_annotations = []
    all_attributes = []

    for _, row in df.iterrows():
        all_annotations.extend(row[annot_col])
        all_attributes.extend(row[attribute_col])

    return all_annotations, all_attributes

In [7]:
for y in ["annotatorAge","annotatorRace", "annotatorGender"]:
    annotations, attributes = extract_annotations_and_attributes(df, "racism", y)
    stat = aposteriori.aposteriori_unimodality(all_annotations=annotations, annotator_group=attributes, comment_group="tweet")
    print(f"{y}: {stat:0.5f}")

annotatorAge: 0.00859
annotatorRace: 0.99357
annotatorGender: 0.91683
